### **Load data & preprocessing**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
import numpy as np
!pip install gdown
import gdown

file_id = "1T3PLuDjjT2ZE8BvXQmU97Rc9wHLrY6CZ"
url = f"https://drive.google.com/uc?id={file_id}"
output = "review_bali.csv"
gdown.download(url, output, quiet=False)
data = pd.read_csv("review_bali.csv")
data.head()

In [ ]:
data = data.dropna()
print(data.isnull().sum())

**Embedding**

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Ambil kolom teks
texts = data["ulasan"].astype(str).tolist()

# Load model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Generate embedding
embeddings = model.encode(texts)

# Simpan sebagai list dalam kolom baru
data["embedding_text"] = embeddings.tolist()

print(data.head())


In [ ]:
print(data.head())

In [ ]:
# --------------------------------------
# 1. Import Library
# --------------------------------------
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import pandas as pd
import requests
from io import BytesIO
import numpy as np
from tqdm import tqdm

# --------------------------------------
# 3. Load Model & Processor
# --------------------------------------
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224')

# --------------------------------------
# 4. Fungsi untuk dapatkan embedding gambar
# --------------------------------------
def get_image_embedding_from_url(url):
    try:
        # Ambil gambar dari URL
        response = requests.get(url, timeout=10)
        image = Image.open(BytesIO(response.content)).convert("RGB")

        # Preprocessing gambar
        inputs = processor(images=image, return_tensors="pt")

        # Dapatkan output dari model
        with torch.no_grad():
            outputs = model(**inputs)

        # Ambil [CLS] token embedding (biasanya di index 0)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        return embedding

    except Exception as e:
        print(f"Error pada {url}: {e}")
        return np.zeros(model.config.hidden_size)

# --------------------------------------
# 5. Generate Embedding untuk semua gambar
# --------------------------------------
image_embeddings = []

for url in tqdm(data["foto_url"].tolist(), desc="Processing images"):
    emb = get_image_embedding_from_url(url)
    image_embeddings.append(emb)

image_embeddings = np.array(image_embeddings)
print("Shape image embedding:", image_embeddings.shape)

# --------------------------------------
# 7. Tambahkan ke dataframe
# --------------------------------------
data["image_embedding"] = image_embeddings.tolist()
print(data.head())

In [ ]:
df = pd.read_pickle("data_with_embeddings.pkl")
df.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np

# Contoh variabel
# text_embeddings, image_embeddings, multimodal_embeddings

# Simpan ke folder tertentu di Google Drive
np.save("/content/drive/MyDrive/FP SML/text_embeddings.npy", embedding_text)
np.save("/content/drive/MyDrive/FP SML/image_embeddings.npy", image_embeddings)

print("✅ Semua file embeddings tersimpan di Google Drive!")

In [ ]:
import pandas as pd
import numpy as np
!pip install gdown
import gdown

file_id = "1slEkvOUUukFw4qeZWIoKym69FwpnJmEn"
url = f"https://drive.google.com/uc?id={file_id}"
output = "review.csv"
gdown.download(url, output, quiet=False)
df = pd.read_csv("review.csv")
df.head()

In [ ]:
embedding_text = np.array(embedding_text)
image_embeddings = np.array(image_embeddings)

print("Text embedding shape:", embedding_text.shape)
print("Image embedding shape:", image_embeddings.shape)

In [ ]:
data.head()

In [ ]:
import numpy as np

text_embeddings = np.load("/content/drive/MyDrive/FP SML/text_embeddings.npy", allow_pickle=True)       # shape (n, 384)
image_embeddings = np.load("/content/drive/MyDrive/FP SML/image_embeddings.npy", allow_pickle=True)   # shape (n, 768)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(text_embeddings[:5])
print(image_embeddings[:5])

In [ ]:
print("Data shape:", data.shape)
print("Embeddings shape:", text_embeddings.shape)

In [ ]:
# pastikan semua embedding jadi numpy array numerik
text_embeddings = np.array(text_embeddings.tolist() if isinstance(text_embeddings, np.ndarray) else text_embeddings)
image_embeddings = np.array(image_embeddings.tolist() if isinstance(image_embeddings, np.ndarray) else image_embeddings)

print("Text dtype:", text_embeddings.dtype)
print("Image dtype:", image_embeddings.dtype)

# sekarang normalisasi akan aman
text_norm = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)
image_norm = image_embeddings / np.linalg.norm(image_embeddings, axis=1, keepdims=True)

# lalu gabungkan
multimodal_embeddings = np.concatenate([text_norm, image_norm], axis=1)
print("✅ Multimodal embedding shape:", multimodal_embeddings.shape)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np

# Contoh variabel
# text_embeddings, image_embeddings, multimodal_embeddings

# Simpan ke folder tertentu di Google Drive
np.save("/content/drive/MyDrive/FP SML/multimodal_embeddings.npy", multimodal_embeddings)

### **Rating Label**




In [ ]:
# --- (1) Ekstrak angka dari teks rating ---
# Contoh: "5 bintang" -> 5
data["rating_num"] = data["rating"].str.extract(r'(\d+)').astype(float)

# --- (2) Tentukan label sentimen ---
conditions = [
    data["rating_num"].isin([1, 2]),
    data["rating_num"] == 3,
    data["rating_num"].isin([4, 5])
]
choices = ["negative", "neutral", "positive"]

data["rating_label"] = np.select(conditions, choices, default="unknown")

# --- (3) Cek hasil ---
print(data[["rating", "rating_num", "rating_label"]].head())


In [ ]:
print(data["rating_label"].value_counts())

import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x="rating_label", data=data, order=["positive", "neutral", "negative"])
plt.title("Distribusi Sentimen Berdasarkan Rating Label")
plt.xlabel("Sentiment")
plt.ylabel("Jumlah Ulasan")
plt.show()

### **Translate**

In [ ]:
!pip install deep-translator

In [ ]:
from deep_translator import GoogleTranslator
import pandas as pd

# misal data kamu
# data = pd.read_csv("review_bali.csv")

# terjemahkan kolom "ulasan" dari Indonesia → Inggris
def translate_to_en(text):
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except:
        return ""

data["ulasan_en"] = data["ulasan"].astype(str).apply(translate_to_en)
print(data[["ulasan", "ulasan_en"]].head())

In [ ]:
data.head()

### **Vader Label**

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

analyzer = SentimentIntensityAnalyzer()

# Pastikan kolom ulasan berupa teks
data["ulasan_en"] = data["ulasan_en"].astype(str)

# Fungsi untuk analisis VADER
def get_vader_sentiment(text):
    scores = analyzer.polarity_scores(text)
    compound = scores["compound"]
    if compound >= 0.05:
        return "positive"
    elif compound <= -0.05:
        return "negative"
    else:
        return "neutral"

# Terapkan ke seluruh kolom
data["vader_score"] = data["ulasan_en"].apply(lambda x: analyzer.polarity_scores(x)["compound"])
data["vader_label"] = data["ulasan_en"].apply(get_vader_sentiment)

print("✅ Analisis selesai!")
print(data.head())

In [ ]:
data.head()

In [ ]:
print(data["vader_label"].value_counts())

import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x="vader_label", data=data, order=["positive", "neutral", "negative"])
plt.title("Distribusi Sentimen Berdasarkan VADER")
plt.xlabel("Sentiment")
plt.ylabel("Jumlah Ulasan")
plt.show()

### **Text Blob**

In [ ]:
from textblob import TextBlob

def textblob_label(text):
    score = TextBlob(str(text)).sentiment.polarity
    if score > 0.05:
        return "positive"
    elif score < -0.05:
        return "negative"
    else:
        return "neutral"

data["textblob_label"] = data["ulasan_en"].apply(textblob_label)

In [ ]:
data.head()

In [ ]:
print(data["textblob_label"].value_counts())

import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x="textblob_label", data=data, order=["positive", "neutral", "negative"])
plt.title("Distribusi Sentimen Berdasarkan Textblob")
plt.xlabel("Sentiment")
plt.ylabel("Jumlah Ulasan")
plt.show()

### **Actual Label**

In [ ]:
import pandas as pd

def majority_label(row):
    labels = [row["rating_label"], row["vader_label"], row["textblob_label"]]
    unique_labels = set(labels)

    # Jika semua label sama → ambil salah satunya
    if len(unique_labels) == 1:
        return labels[0]

    # Hitung kemunculan tiap label
    counts = pd.Series(labels).value_counts()
    top_label = counts.idxmax()

    # Jika ada 3 label berbeda → netral
    if len(unique_labels) == 3:
        return "neutral"

    # Kalau tidak, ambil label yang paling sering muncul
    return top_label

# Terapkan ke DataFrame
data["label_aktual"] = data.apply(majority_label, axis=1)

print(data[["rating_label", "vader_label", "textblob_label", "label_aktual"]].head())

In [ ]:
print(data["label_aktual"].value_counts())

import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x="label_aktual", data=data, order=["positive", "neutral", "negative"])
plt.title("Distribusi Sentimen Berdasarkan VADER")
plt.xlabel("Sentiment")
plt.ylabel("Jumlah Ulasan")
plt.show()

In [ ]:
data.head(10)

In [ ]:
import pandas as pd

# ganti nama dan path sesuai kebutuhan
save_path = "/content/drive/MyDrive/FP SML/hasil_label_sentimen.csv"

# simpan ke CSV
data.to_csv(save_path, index=False, encoding="utf-8")

print(f"✅ File berhasil disimpan ke Google Drive:\n{save_path}")

**Data Label Aktual (sementara)**

In [ ]:
import pandas as pd
import numpy as np
!pip install gdown
import gdown

file_id = "1T4NKsxemv1QxpuDjZI84qyzH0rWKyfes"
url = f"https://drive.google.com/uc?id={file_id}"
output = "hasil_label_sentimen.csv"
gdown.download(url, output, quiet=False)
datalabel = pd.read_csv("hasil_label_sentimen.csv")
datalabel.head()

### **Labeling Afinn**

In [ ]:
!pip install afinn

In [ ]:
from afinn import Afinn
import pandas as pd

# inisialisasi AFINN analyzer
afn = Afinn()

In [ ]:
def afinn_sentiment_label(text):
    try:
        score = afn.score(str(text))
        if score > 0:
            label = "positive"
        elif score < 0:
            label = "negative"
        else:
            label = "neutral"
        return pd.Series({"afinn_score": score, "afinn_label": label})
    except:
        return pd.Series({"afinn_score": 0, "afinn_label": "neutral"})

In [ ]:
datalabel[["afinn_score", "afinn_label"]] = datalabel["ulasan_en"].astype(str).apply(afinn_sentiment_label)

print("✅ Labeling selesai!")
print(datalabel[["ulasan_en", "afinn_score", "afinn_label"]].head())

In [ ]:
datalabel.head()

In [ ]:
print(datalabel["afinn_label"].value_counts())

import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x="afinn_label", data=datalabel, order=["positive", "neutral", "negative"])
plt.title("Distribusi Sentimen Berdasarkan afinn")
plt.xlabel("Sentiment")
plt.ylabel("Jumlah Ulasan")
plt.show()

In [ ]:
datalabel = datalabel.drop(columns=["label_aktual"])
datalabel.head()

### **Bentuk Label Aktual**
vader, textblob, afinn

In [ ]:
import pandas as pd

def majority_label(row):
    labels = [row["vader_label"], row["textblob_label"], row["afinn_label"]]
    unique_labels = set(labels)

    # Jika semua label sama → ambil salah satunya
    if len(unique_labels) == 1:
        return labels[0]

    # Hitung kemunculan tiap label
    counts = pd.Series(labels).value_counts()
    top_label = counts.idxmax()

    # Jika ada 3 label berbeda → netral
    if len(unique_labels) == 3:
        return "neutral"

    # Kalau tidak, ambil label yang paling sering muncul
    return top_label

# Terapkan ke DataFrame
datalabel["label_aktual"] = datalabel.apply(majority_label, axis=1)

datalabel[["rating_label", "vader_label", "textblob_label", "label_aktual"]].head()

In [ ]:
import pandas as pd

# ganti nama dan path sesuai kebutuhan
save_path = "/content/drive/MyDrive/Colab Notebooks/SML/data_label.csv"

# simpan ke CSV
datalabel.to_csv(save_path, index=False, encoding="utf-8")

print(f"✅ File berhasil disimpan ke Google Drive:\n{save_path}")

### **Panggil Data Label Aktual 3 methods**

In [ ]:
import pandas as pd
import numpy as np
!pip install gdown
import gdown

file_id = "13oy1pH836mylOn5h6Itp3nGvFc51D64i"
url = f"https://drive.google.com/uc?id={file_id}"
output = "label_fix.csv"
gdown.download(url, output, quiet=False)
datalabelfix = pd.read_csv("label_fix.csv")
datalabelfix.head()

In [ ]:
print(datalabelfix["label_aktual"].value_counts())

import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x="label_aktual", data=datalabelfix, order=["positive", "neutral", "negative"])
plt.title("Distribusi Sentimen Berdasarkan label aktual")
plt.xlabel("Sentiment")
plt.ylabel("Jumlah Ulasan")
plt.show()

### **Panggil Data embedding text, image, multimodal**

In [ ]:
file_id_text = "1XgJGrLUhwQWrz6yV4N-GS1XufGcE11y8"        # 🔁 Ganti ID dengan ID file kamu
file_id_image = "1nwQpap8dVugKC7ptj7e-4RK3UjyoSAP3"
file_id_mult = "1wmLmZR8xDed_gBHAy5FxYBtuBCb9MgN_"

url_text = f"https://drive.google.com/uc?id={file_id_text}"
url_image = f"https://drive.google.com/uc?id={file_id_image}"
url_mult = f"https://drive.google.com/uc?id={file_id_mult}"

gdown.download(url_text, "text_embeddings.npy", quiet=False)
gdown.download(url_image, "image_embeddings.npy", quiet=False)
gdown.download(url_mult, "multimodal_embeddings.npy", quiet=False)

text_embeddings = np.load("text_embeddings.npy", allow_pickle=True)
image_embeddings = np.load("image_embeddings.npy", allow_pickle=True)
multimodal_embeddings = np.load("multimodal_embeddings.npy", allow_pickle=True)

print("text_embeddings:", np.array(text_embeddings).shape)
print("image_embeddings:", np.array(image_embeddings).shape)
print("multimodal_embeddings:", np.array(multimodal_embeddings).shape)